# INTRODUCTION

In this part of assignment I will show my skill to work with raw data, explore and analyze dataset, and cluster it using K-Means Clustering algorithm and Foursquare API.

First, let's download all libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Download and explore dataset

In [5]:
#download raw data
raw_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(raw_data.content, 'html.parser')

#soup

In [23]:
#parce raw data
table = soup.find('table', {"class":"wikitable sortable"})
#create table
all_rows = table.find_all('tr')
all_rows = all_rows[1:]
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df=pd.DataFrame(columns=column_names)
for row in all_rows:
    row = row.text.split('\n')
    #If cell has borough and not assigned neighborhood, then the neighborhood will have its own value of row[3] (borough)
    if row[3] == "Not assigned":
        neighborhood=row[2]
    else:
        neighborhood=row[3]
    df = df.append({'PostalCode': row[1], 'Borough': row[2], 'Neighborhood': neighborhood}, ignore_index=True)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


let's see initial size of our dataframe

In [24]:
df.shape

(287, 3)

Then we exclude not assigned boroughs and examine size of new dataframe

In [25]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(210, 3)

Now we have to combine neighborhoods with the same postal code

In [26]:
df=df.groupby(['PostalCode']).agg({'Borough':(lambda x: x.unique()), 'Neighborhood':(lambda x: ', '.join(x))}).reset_index()
                                                                     
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let's see final size of our dataframe

In [27]:
df.shape

(103, 3)